In [ ]:
from flask import Flask, request, render_template, flash, redirect, url_for, send_from_directory
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
from joblib import load
from werkzeug.utils import secure_filename
import os
import logging
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from matplotlib.gridspec import GridSpec

# Configure logging
logging.basicConfig(level=logging.INFO)
logging.getLogger('matplotlib.font_manager').setLevel(logging.INFO)

# Flask app
app = Flask(__name__)
app.secret_key = 'your_secret_key_here'
app.config['UPLOAD_FOLDER'] = 'Uploads'
app.config['REPORTS_FOLDER'] = r"D:/upworkhealt/reports"
app.config['STATIC_IMAGES'] = 'static/images'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB limit

# Paths for data, model, scaler, and threshold
DATA_PATH = r"D:/upworkhealt/data"
MODEL_PATH = r"C:/Users/HP/Desktop/upworkhealt/saved_models/enhanced_hybrid_cnn_lstm_final.keras"
SCALER_PATH = r"C:/Users/HP/Desktop/upworkhealt/saved_models/enhanced_scaler.save"
THRESHOLD_PATH = r"C:/Users/HP/Desktop/upworkhealt/saved_models/optimal_threshold.txt"

# Ensure folders exist
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
os.makedirs(app.config['REPORTS_FOLDER'], exist_ok=True)
os.makedirs(app.config['STATIC_IMAGES'], exist_ok=True)

# Create a placeholder pdf-preview.png if it doesn't exist
preview_image_path = os.path.join(app.config['STATIC_IMAGES'], 'pdf-preview.png')
if not os.path.exists(preview_image_path):
    plt.figure(figsize=(4, 3))
    plt.text(0.5, 0.5, 'ECG Report Preview', ha='center', va='center', fontsize=12)
    plt.axis('off')
    plt.savefig(preview_image_path, bbox_inches='tight')
    plt.close()
    logging.info(f"Created placeholder pdf-preview.png at {preview_image_path}")

# Load the saved Keras model (Hybrid CNN-LSTM)
try:
    model = tf.keras.models.load_model(MODEL_PATH)
    logging.info(f"✅ Hybrid CNN-LSTM model loaded successfully from {MODEL_PATH}")
except Exception as e:
    logging.error(f"❌ Failed to load Hybrid CNN-LSTM model: {e}")
    model = None

# Load the saved scaler
try:
    scaler = load(SCALER_PATH)
    logging.info(f"✅ Scaler loaded successfully from {SCALER_PATH}")
except Exception as e:
    logging.error(f"❌ Failed to load scaler: {e}")
    scaler = StandardScaler()

# Load the optimal threshold
try:
    with open(THRESHOLD_PATH, 'r') as f:
        optimal_threshold = float(f.read().strip())
    logging.info(f"✅ Optimal threshold loaded: {optimal_threshold}")
except Exception as e:
    logging.warning(f"❌ Failed to load optimal threshold, using default 0.5: {e}")
    optimal_threshold = 0.5

# Label mapping for binary classification
label_mapping = {
    0: 'Normal',
    1: 'Abnormal'
}

# Detailed descriptions for each classification
classification_descriptions = {
    'Normal': {
        'description': "The ECG appears to be within normal limits, showing regular sinus rhythm with normal P waves, QRS complexes, and T waves.",
        'recommendation': "No immediate medical intervention is required based on this ECG. However, regular check-ups are recommended.",
        'clinical_notes': "Normal ECG findings suggest a low probability of acute cardiac pathology at this time."
    },
    'Abnormal': {
        'description': "The ECG shows abnormalities that may indicate potential cardiac issues. This could include arrhythmias, conduction abnormalities, or signs of ischemia.",
        'recommendation': "Consult with a cardiologist for further evaluation. Additional diagnostic tests may be required.",
        'clinical_notes': "Abnormal ECG findings should be correlated with clinical symptoms and patient history for proper diagnosis."
    }
}

def preprocess_data(data):
    try:
        if data.ndim == 1:
            data = data.reshape(1, -1)
        elif data.ndim > 2:
            raise ValueError(f"Unexpected data shape: {data.shape}")

        if data.shape[1] != 187:
            raise ValueError(f"Expected 187 features, got {data.shape[1]}")

        data = scaler.transform(data)
        data = data.reshape(data.shape[0], 187, 1)
        logging.info(f"Data preprocessed successfully, shape: {data.shape}")
        return data
    except Exception as e:
        logging.error(f"Error in preprocess_data: {str(e)}")
        raise

def generate_report_id():
    return datetime.now().strftime("ECG%Y%m%d%H%M%S")

def generate_pdf_report(report_id, class_name, confidence, raw_data):
    filename = f'report_{report_id}.pdf'
    report_path = os.path.join(app.config['REPORTS_FOLDER'], filename).replace('\\', '/')

    try:
        plt.style.use('seaborn')
        sns.set_palette("husl")
        
        # Get detailed description Ark of classification_descriptions
        class_info = classification_descriptions.get(class_name, {
            'description': 'No specific description available.',
            'recommendation': 'Consult with a healthcare professional.',
            'clinical_notes': 'Further evaluation may be necessary.'
        })

        with PdfPages(report_path) as pdf:
            # Create title page
            fig = plt.figure(figsize=(11, 8.5))
            gs = GridSpec(2, 1, figure=fig, height_ratios=[3, 1])
            
            # Title section
            ax1 = fig.add_subplot(gs[0])
            ax1.text(0.5, 0.7, 'ELECTROCARDIOGRAM (ECG) ANALYSIS REPORT', 
                    ha='center', va='center', fontsize=18, fontweight='bold')
            ax1.text(0.5, 0.6, f'Classification: {class_name}', 
                    ha='center', va='center', fontsize=16, color='blue' if class_name == 'Normal' else 'red')
            ax1.text(0.5, 0.5, f'Confidence Level: {confidence:.2f}%', 
                    ha='center', va='center', fontsize=14)
            ax1.text(0.5, 0.4, f'Report ID: {report_id}', 
                    ha='center', va='center', fontsize=12)
            ax1.text(0.5, 0.3, f'Date of Analysis: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}', 
                    ha='center', va='center', fontsize=12)
            ax1.axis('off')
            
            # Footer section
            ax2 = fig.add_subplot(gs[1])
            ax2.text(0.5, 0.7, 'This report is generated by automated ECG analysis system.', 
                    ha='center', va='center', fontsize=10, style='italic')
            ax2.text(0.5, 0.3, 'For medical use only - Consult a cardiologist for clinical interpretation.', 
                    ha='center', va='center', fontsize=10, style='italic')
            ax2.axis('off')
            
            pdf.savefig(fig)
            plt.close(fig)

            # Create detailed report page
            fig = plt.figure(figsize=(11, 8.5))
            gs = GridSpec(3, 1, figure=fig, height_ratios=[1, 1, 2])
            
            # Classification details
            ax1 = fig.add_subplot(gs[0])
            ax1.text(0, 0.9, 'ECG FINDINGS:', fontweight='bold', fontsize=12)
            ax1.text(0.1, 0.7, class_info['description'], fontsize=10, wrap=True)
            ax1.text(0, 0.5, 'CLINICAL NOTES:', fontweight='bold', fontsize=12)
            ax1.text(0.1, 0.3, class_info['clinical_notes'], fontsize=10, wrap=True)
            ax1.text(0, 0.1, 'RECOMMENDATIONS:', fontweight='bold', fontsize=12)
            ax1.text(0.1, -0.1, class_info['recommendation'], fontsize=10, wrap=True)
            ax1.axis('off')
            
            # ECG signal plot
            ax2 = fig.add_subplot(gs[1:])
            signal = raw_data[0, :] if raw_data.ndim == 2 else raw_data
            ax2.plot(signal, color='blue', linewidth=1.0)
            ax2.set_title(f'ECG Signal Waveform\nClassification: {class_name} (Confidence: {confidence:.2f}%)', fontsize=12)
            ax2.set_xlabel('Time (ms)')
            ax2.set_ylabel('Amplitude (mV)')
            ax2.grid(True, alpha=0.3)
            
            # Add annotations to the ECG plot
            if class_name == 'Normal':
                ax2.text(0.05, 0.95, 'Normal Sinus Rhythm', 
                        transform=ax2.transAxes, color='green', 
                        bbox=dict(facecolor='white', alpha=0.8))
            else:
                ax2.text(0.05, 0.95, 'Abnormal Findings Detected', 
                        transform=ax2.transAxes, color='red', 
                        bbox=dict(facecolor='white', alpha=0.8))
            
            plt.tight_layout()
            pdf.savefig(fig)
            plt.close(fig)

            # Add a page with technical details
            fig = plt.figure(figsize=(11, 8.5))
            gs = GridSpec(2, 1, figure=fig)
            
            # Technical details
            ax1 = fig.add_subplot(gs[0])
            ax1.text(0, 0.9, 'TECHNICAL DETAILS:', fontweight='bold', fontsize=12)
            ax1.text(0.1, 0.7, f'Analysis Model: Enhanced Hybrid CNN-LSTM Deep Learning Model', fontsize=10)
            ax1.text(0.1, 0.6, f'Input Features: 187 time-series points (standardized)', fontsize=10)
            ax1.text(0.1, 0.5, f'Classification Threshold: {optimal_threshold:.2f} probability', fontsize=10)
            ax1.text(0.1, 0.4, f'Model Confidence: {confidence:.2f}%', fontsize=10)
            ax1.text(0.1, 0.3, f'Report Generation Time: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}', fontsize=10)
            ax1.axis('off')
            
            # Disclaimer
            ax2 = fig.add_subplot(gs[1])
            ax2.text(0, 0.9, 'IMPORTANT DISCLAIMER:', fontweight='bold', fontsize=12, color='red')
            ax2.text(0.1, 0.7, 'This automated ECG analysis is intended to assist healthcare professionals and should not be used as the sole basis for clinical decisions.', fontsize=10, wrap=True)
            ax2.text(0.1, 0.5, 'The interpretation may not detect all abnormalities and may occasionally produce false results.', fontsize=10, wrap=True)
            ax2.text(0.1, 0.3, 'Always correlate with clinical findings, patient history, and other diagnostic tests.', fontsize=10, wrap=True)
            ax2.text(0.1, 0.1, 'For any concerns or abnormal findings, please consult a qualified cardiologist.', fontsize=10, wrap=True)
            ax2.axis('off')
            
            pdf.savefig(fig)
            plt.close(fig)

        logging.info(f"PDF report generated successfully: {report_path}")
        return filename
    except Exception as e:
        logging.error(f"Error generating PDF report: {str(e)}")
        raise

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'ecgfile' not in request.files:
        flash('لم يتم رفع أي ملف', 'error')
        return redirect(url_for('index'))

    file = request.files['ecgfile']
    if file.filename == '':
        flash('لم يتم اختيار ملف', 'error')
        return redirect(url_for('index'))

    if not file.filename.lower().endswith('.csv'):
        flash('الرجاء رفع ملف CSV', 'error')
        return redirect(url_for('index'))

    try:
        filename = secure_filename(file.filename)
        temp_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(temp_path)

        df = pd.read_csv(temp_path, header=None)
        os.remove(temp_path)
        data = df.values

        if data.ndim == 1:
            data = data.reshape(1, -1)

        if data.shape[1] != 187:
            flash(f'تنسيق الملف غير صالح: يجب أن يحتوي على 187 قيمة، تم العثور على {data.shape[1]}', 'error')
            return redirect(url_for('index'))

        processed_data = preprocess_data(data)

        if model is None:
            flash('النموذج غير متوفر', 'error')
            return redirect(url_for('index'))

        prediction_prob = model.predict(processed_data)
        prediction = (prediction_prob > optimal_threshold).astype(int).flatten()[0]
        class_name = label_mapping.get(prediction, 'غير معروف')
        confidence = float(prediction_prob[0][0]) * 100 if prediction == 1 else (1.0 - float(prediction_prob[0][0])) * 100

        report_id = generate_report_id()
        report_filename = generate_pdf_report(report_id, class_name, confidence, data)

        return render_template('result.html', 
                               prediction=class_name, 
                               confidence=f"{confidence:.2f}%",
                               class_index=prediction,
                               now=datetime.now(),
                               report_id=report_id)

    except Exception as e:
        flash(f'خطأ في معالجة الملف: {str(e)}', 'error')
        return redirect(url_for('index'))

@app.route('/view-report/<report_id>')
def view_report(report_id):
    filename = f'report_{report_id}.pdf'
    report_path = os.path.join(app.config['REPORTS_FOLDER'], filename).replace('\\', '/')

    if not os.path.exists(report_path):
        flash(f'Report {report_id} not found', 'error')
        return redirect(url_for('index'))

    try:
        return send_from_directory(
            app.config['REPORTS_FOLDER'],
            filename,
            mimetype='application/pdf'
        )
    except Exception as e:
        flash(f'Error serving report: {str(e)}', 'error')
        return redirect(url_for('index'))

@app.route('/download-report/<report_id>')
def download_report(report_id):
    filename = f'report_{report_id}.pdf'
    report_path = os.path.join(app.config['REPORTS_FOLDER'], filename).replace('\\', '/')

    if not os.path.exists(report_path):
        flash(f'Report {report_id} not found', 'error')
        return redirect(url_for('index'))

    try:
        return send_from_directory(
            app.config['REPORTS_FOLDER'],
            filename,
            as_attachment=True,
            download_name=f'ECG_Report_{report_id}.pdf'
        )
    except Exception as e:
        flash(f'Error downloading report: {str(e)}', 'error')
        return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

INFO:root:✅ Hybrid CNN-LSTM model loaded successfully from C:/Users/HP/Desktop/upworkhealt/saved_models/enhanced_hybrid_cnn_lstm_final.keras
INFO:root:✅ Scaler loaded successfully from C:/Users/HP/Desktop/upworkhealt/saved_models/enhanced_scaler.save
INFO:root:✅ Optimal threshold loaded: 0.7146816


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [26/Jun/2025 19:58:06] "GET / HTTP/1.1" 200 -
INFO:root:Data preprocessed successfully, shape: (1, 187, 1)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


INFO:root:PDF report generated successfully: D:/upworkhealt/reports/report_ECG20250626195815.pdf
INFO:werkzeug:127.0.0.1 - - [26/Jun/2025 19:58:16] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jun/2025 19:59:14] "GET / HTTP/1.1" 200 -
INFO:root:Data preprocessed successfully, shape: (1, 187, 1)


...

INFO:root:PDF report generated successfully: D:/upworkhealt/reports/report_ECG20250626200049.pdf
INFO:werkzeug:127.0.0.1 - - [26/Jun/2025 20:00:50] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jun/2025 20:06:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jun/2025 20:08:11] "GET / HTTP/1.1" 200 -
INFO:root:Data preprocessed successfully, shape: (1, 187, 1)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step


INFO:root:PDF report generated successfully: D:/upworkhealt/reports/report_ECG20250626200835.pdf
INFO:werkzeug:127.0.0.1 - - [26/Jun/2025 20:08:36] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jun/2025 20:08:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jun/2025 20:08:51] "GET /sample/other HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Jun/2025 20:08:56] "GET /sample/afib HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Jun/2025 20:09:01] "GET /sample/normal HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Jun/2025 20:09:13] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jun/2025 20:12:00] "POST /predict HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [26/Jun/2025 20:12:00] "GET / HTTP/1.1" 200 -
INFO:root:Data preprocessed successfully, shape: (1, 187, 1)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step


INFO:root:PDF report generated successfully: D:/upworkhealt/reports/report_ECG20250626201310.pdf
INFO:werkzeug:127.0.0.1 - - [26/Jun/2025 20:13:11] "POST /predict HTTP/1.1" 200 -
